041: データを読み込んで利用データを整形する

In [1]:
import pandas as pd
customer = pd.read_csv('./input/03/customer_join.csv')
use_log_montrhs = pd.read_csv('./input/04/use_log_months.csv')

In [2]:
print(customer)
print(use_log_montrhs)

     customer_id   name class gender  start_date end_date campaign_id  \
0       OA832399   XXXX   C01      F  2015-05-01      NaN         CA1   
1       PL270116  XXXXX   C01      M  2015-05-01      NaN         CA1   
2       OA974876  XXXXX   C01      M  2015-05-01      NaN         CA1   
3       HD024127  XXXXX   C01      F  2015-05-01      NaN         CA1   
4       HD661448  XXXXX   C03      F  2015-05-01      NaN         CA1   
...          ...    ...   ...    ...         ...      ...         ...   
4187    HD676663   XXXX   C01      M  2019-03-14      NaN         CA1   
4188    HD246549  XXXXX   C01      F  2019-03-14      NaN         CA1   
4189    GD037007  XXXXX   C03      M  2019-03-14      NaN         CA1   
4190    OA953150  XXXXX   C01      M  2019-03-14      NaN         CA1   
4191    IK692635  XXXXX   C02      F  2019-03-15      NaN         CA1   

      is_deleted campaign_name class_name  price  count  routine_flag  \
0              0            通常     オールタイム  10500  

当月と1ヶ月前のデータから来月の退会を予測する

In [3]:
year_month = list(use_log_montrhs['年月'].unique())
use_log = pd.DataFrame()
for i in range(1, len(year_month)):
    tmp = use_log_montrhs.loc[use_log_montrhs['年月'] == year_month[i]]
    tmp.rename(columns={'count':'count_0'}, inplace=True)
    tmp_before = use_log_montrhs.loc[use_log_montrhs['年月']==year_month[i-1]]
    del tmp_before['年月']
    tmp_before.rename(columns={'count':'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    use_log = pd.concat([use_log, tmp], ignore_index=True)
print(use_log)


           年月 customer_id  count_0  count_1
0      201805    AS002855        5      4.0
1      201805    AS009373        4      3.0
2      201805    AS015233        7      NaN
3      201805    AS015315        3      6.0
4      201805    AS015739        5      7.0
...       ...         ...      ...      ...
33846  201903    TS995853        8     11.0
33847  201903    TS998593        8      7.0
33848  201903    TS999079        3      2.0
33849  201903    TS999231        6      6.0
33850  201903    TS999855        4      4.0

[33851 rows x 4 columns]


/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


042: 退会前月の退会顧客データを作成する

In [4]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer['is_deleted']==1]
exit_customer['exit_date'] = None
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)
#exit_customer['exit_date'] = pd.to_datetime(exit_customer['exit_date'])
exit_customer['年月'] = pd.to_datetime(exit_customer['exit_date']).dt.strftime("%Y%m")
use_log['年月'] = use_log['年月'].astype(str)
exit_use_log = pd.merge(use_log, exit_customer, on=['customer_id', '年月'], how='left')

print(len(exit_use_log))
print(exit_use_log)

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


33851
           年月 customer_id  count_0  count_1 name class gender start_date  \
0      201805    AS002855        5      4.0  NaN   NaN    NaN        NaN   
1      201805    AS009373        4      3.0  NaN   NaN    NaN        NaN   
2      201805    AS015233        7      NaN  NaN   NaN    NaN        NaN   
3      201805    AS015315        3      6.0  NaN   NaN    NaN        NaN   
4      201805    AS015739        5      7.0  NaN   NaN    NaN        NaN   
...       ...         ...      ...      ...  ...   ...    ...        ...   
33846  201903    TS995853        8     11.0  NaN   NaN    NaN        NaN   
33847  201903    TS998593        8      7.0  NaN   NaN    NaN        NaN   
33848  201903    TS999079        3      2.0  NaN   NaN    NaN        NaN   
33849  201903    TS999231        6      6.0  NaN   NaN    NaN        NaN   
33850  201903    TS999855        4      4.0  NaN   NaN    NaN        NaN   

      end_date campaign_id  ...  price count routine_flag  mean  median  max  \
0

データが欠損している行は削除して、欠損値がないデータのみを残す

In [5]:
exit_use_log = exit_use_log.dropna(subset=['name'])
print(len(exit_use_log))
print(len(exit_use_log['customer_id'].unique()))
print(exit_use_log)

1104
1104
           年月 customer_id  count_0  count_1    name class gender  start_date  \
19     201805    AS055680        3      3.0   XXXXX   C01      M  2018-03-01   
57     201805    AS169823        2      3.0      XX   C01      M  2017-11-01   
110    201805    AS305860        5      3.0    XXXX   C01      M  2017-06-01   
128    201805    AS363699        5      3.0   XXXXX   C01      M  2018-02-01   
147    201805    AS417696        1      4.0      XX   C03      F  2017-09-01   
...       ...         ...      ...      ...     ...   ...    ...         ...   
30758  201902    TS645212        4      2.0    XXXX   C03      F  2018-03-01   
30787  201902    TS741703        5      6.0    XXXX   C03      M  2018-12-08   
30827  201902    TS859258        1      3.0   XXXXX   C02      F  2018-12-07   
30842  201902    TS886985        5      3.0     XXX   C02      F  2018-03-01   
30852  201902    TS921837        2      3.0  XXXXXX   C01      M  2018-06-04   

        end_date campaign_id 

043: 継続顧客のデータを作成する

In [6]:
conti_customer = customer.loc[customer['is_deleted']==0]
conti_use_log = pd.merge(use_log, conti_customer, on=['customer_id'], how='left')
print(len(conti_use_log))
print(conti_use_log)
conti_use_log = conti_use_log.dropna(subset=['name'])
print(len(conti_use_log))
print(conti_use_log)

33851
           年月 customer_id  count_0  count_1    name class gender  start_date  \
0      201805    AS002855        5      4.0    XXXX   C03      F  2016-11-01   
1      201805    AS009373        4      3.0      XX   C01      F  2015-11-01   
2      201805    AS015233        7      NaN   XXXXX   C01      M  2018-05-13   
3      201805    AS015315        3      6.0   XXXXX   C01      M  2015-07-01   
4      201805    AS015739        5      7.0   XXXXX   C03      M  2017-06-01   
...       ...         ...      ...      ...     ...   ...    ...         ...   
33846  201903    TS995853        8     11.0    XXXX   C01      M  2019-02-08   
33847  201903    TS998593        8      7.0   XXXXX   C03      M  2018-09-01   
33848  201903    TS999079        3      2.0     XXX   C03      M  2016-06-01   
33849  201903    TS999231        6      6.0    XXXX   C01      M  2017-03-01   
33850  201903    TS999855        4      4.0  XXXXXX   C01      F  2017-04-01   

      end_date campaign_id  ...  

退会データ`1104`と継続データ`27422`の不均衡を解消するために、継続データをアンダーサンプリングする。  
継続データをシャッフルして`customer_id`の重複を削除する。シャッフルしなと特定の年月のだけが残っちゃうよ。

In [7]:
conti_use_log = conti_use_log.sample(frac=1).reset_index(drop=True)
conti_use_log = conti_use_log.drop_duplicates(subset=['customer_id'])
print(len(conti_use_log))
print(conti_use_log)

2842
           年月 customer_id  count_0  count_1    name class gender  start_date  \
0      201809    HD552859        7      5.0    XXXX   C03      M  2017-05-01   
1      201903    TS413023        5      5.0      XX   C01      M  2015-11-01   
2      201808    HI650765        4      7.0    XXXX   C01      M  2015-11-01   
3      201810    GD266112        2      4.0     XXX   C03      M  2015-08-01   
4      201808    AS818410        6      3.0    XXXX   C01      M  2017-01-01   
...       ...         ...      ...      ...     ...   ...    ...         ...   
24305  201903    HI785400       11      NaN   XXXXX   C02      F  2019-03-05   
24599  201903    AS076940        7      NaN   XXXXX   C01      F  2019-03-12   
26060  201903    IK473112        5      NaN    XXXX   C01      F  2019-03-06   
26278  201903    HD693171        7      NaN  XXXXXX   C02      F  2019-03-07   
27066  201903    HD246549       10      NaN   XXXXX   C01      F  2019-03-14   

      end_date campaign_id  ...  c

退会顧客と継続顧客を縦に結合する

In [8]:
predict_data = pd.concat([conti_use_log, exit_use_log], ignore_index=True)
print(len(predict_data))
print(predict_data)

3946
          年月 customer_id  count_0  count_1    name class gender  start_date  \
0     201809    HD552859        7      5.0    XXXX   C03      M  2017-05-01   
1     201903    TS413023        5      5.0      XX   C01      M  2015-11-01   
2     201808    HI650765        4      7.0    XXXX   C01      M  2015-11-01   
3     201810    GD266112        2      4.0     XXX   C03      M  2015-08-01   
4     201808    AS818410        6      3.0    XXXX   C01      M  2017-01-01   
...      ...         ...      ...      ...     ...   ...    ...         ...   
3941  201902    TS645212        4      2.0    XXXX   C03      F  2018-03-01   
3942  201902    TS741703        5      6.0    XXXX   C03      M  2018-12-08   
3943  201902    TS859258        1      3.0   XXXXX   C02      F  2018-12-07   
3944  201902    TS886985        5      3.0     XXX   C02      F  2018-03-01   
3945  201902    TS921837        2      3.0  XXXXXX   C01      M  2018-06-04   

                 end_date campaign_id  ...    

変数の追加を行う  
データが時間軸を持ったデータなので、そのデータ時点での在籍期間をデータに追加する。

In [9]:
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['年月'], format="%Y%m")
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years*12 + delta.months)

print(predict_data)


/tmp/ipykernel_88127/2156544981.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data['period'][i] = int(delta.years*12 + delta.months)


          年月 customer_id  count_0  count_1    name class gender start_date  \
0     201809    HD552859        7      5.0    XXXX   C03      M 2017-05-01   
1     201903    TS413023        5      5.0      XX   C01      M 2015-11-01   
2     201808    HI650765        4      7.0    XXXX   C01      M 2015-11-01   
3     201810    GD266112        2      4.0     XXX   C03      M 2015-08-01   
4     201808    AS818410        6      3.0    XXXX   C01      M 2017-01-01   
...      ...         ...      ...      ...     ...   ...    ...        ...   
3941  201902    TS645212        4      2.0    XXXX   C03      F 2018-03-01   
3942  201902    TS741703        5      6.0    XXXX   C03      M 2018-12-08   
3943  201902    TS859258        1      3.0   XXXXX   C02      F 2018-12-07   
3944  201902    TS886985        5      3.0     XXX   C02      F 2018-03-01   
3945  201902    TS921837        2      3.0  XXXXXX   C01      M 2018-06-04   

                 end_date campaign_id  ...  routine_flag      m

欠損値の除去を行う  
結果から、`count_1`と`end_date`、`exit_date`に欠損があることがわかる  
`end_date`と`exit_date`は退会顧客しか値がないので値がないのは正常である。  
`cout_1`が欠損の行だけ削除を行う

In [10]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               257
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
campaign_name           0
class_name              0
price                   0
count                   0
routine_flag            0
mean                    0
median                  0
max                     0
min                     0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [12]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2637
campaign_id             0
is_deleted              0
campaign_name           0
class_name              0
price                   0
count                   0
routine_flag            0
mean                    0
median                  0
max                     0
min                     0
calc_date               0
membership_period       0
exit_date            2637
period                  0
now_date                0
dtype: int64

ここまでの整理  
一ヶ月前の利用回数: `count_1`  
今月の利用回数: `count_0`

In [13]:
traget_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flag', 'period', 'is_deleted']
predict_data = predict_data[traget_col]
print(predict_data)

     campaign_name class_name gender  count_1  routine_flag  period  \
0               通常        ナイト      M      5.0           1.0      16   
1               通常     オールタイム      M      5.0           1.0      40   
2               通常     オールタイム      M      7.0           1.0      33   
3               通常        ナイト      M      4.0           1.0      38   
4               通常     オールタイム      M      3.0           1.0      19   
...            ...        ...    ...      ...           ...     ...   
3941            通常        ナイト      F      2.0           0.0      11   
3942         入会費無料        ナイト      M      6.0           0.0       1   
3943         入会費無料      デイタイム      F      3.0           0.0       1   
3944            通常      デイタイム      F      3.0           1.0      11   
3945         入会費半額     オールタイム      M      3.0           1.0       7   

      is_deleted  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  
...          ...  
3941         1

`get_dummies`を利用して`ワンホットエンコード`を行う

In [14]:
predict_data = pd.get_dummies(predict_data)
print(predict_data)

      count_1  routine_flag  period  is_deleted  campaign_name_入会費半額  \
0         5.0           1.0      16         0.0                    0   
1         5.0           1.0      40         0.0                    0   
2         7.0           1.0      33         0.0                    0   
3         4.0           1.0      38         0.0                    0   
4         3.0           1.0      19         0.0                    0   
...       ...           ...     ...         ...                  ...   
3941      2.0           0.0      11         1.0                    0   
3942      6.0           0.0       1         1.0                    0   
3943      3.0           0.0       1         1.0                    0   
3944      3.0           1.0      11         1.0                    0   
3945      3.0           1.0       7         1.0                    1   

      campaign_name_入会費無料  campaign_name_通常  class_name_オールタイム  \
0                       0                 1                  0   
1  

ワンホットエンコードでは1列は冗長なので削除を行う

In [15]:
del predict_data['gender_M']
del predict_data['class_name_ナイト']
del predict_data['campaign_name_通常']
print(predict_data)

      count_1  routine_flag  period  is_deleted  campaign_name_入会費半額  \
0         5.0           1.0      16         0.0                    0   
1         5.0           1.0      40         0.0                    0   
2         7.0           1.0      33         0.0                    0   
3         4.0           1.0      38         0.0                    0   
4         3.0           1.0      19         0.0                    0   
...       ...           ...     ...         ...                  ...   
3941      2.0           0.0      11         1.0                    0   
3942      6.0           0.0       1         1.0                    0   
3943      3.0           0.0       1         1.0                    0   
3944      3.0           1.0      11         1.0                    0   
3945      3.0           1.0       7         1.0                    1   

      campaign_name_入会費無料  class_name_オールタイム  class_name_デイタイム  gender_F  
0                       0                  0                

いよいよ決定木を使って退会の予測を行う

```
X: 説明変数
y: 目的変数

X_train: 学習用の説明変数
y_train: 学習用の目的変数
X_test: テスト用の説明変数
y_test: テスト用の目的変数
```

In [19]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

# 退会者と継続者を同じ数だけ取得する
exit = predict_data.loc[predict_data['is_deleted'] == 1]
conti = predict_data.loc[predict_data['is_deleted'] == 0].sample(len(exit))
print(exit)
print(conti)

X = pd.concat([exit, conti], ignore_index=True) # 学習の説明変数
y = X['is_deleted']  # 学習用の目的変数
del X['is_deleted']  # 学習データには退会結果はいらないので削除
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)  # データを分割
model = DecisionTreeClassifier(random_state=0)  # モデルをインスタンス化
model.fit(X_train, y_train) # 学習
y_test_pred = model.predict(X_test) # 予測実施
print(y_test_pred)

      count_1  routine_flag  period  is_deleted  campaign_name_入会費半額  \
2842      3.0           0.0       2         1.0                    0   
2843      3.0           1.0       6         1.0                    0   
2844      3.0           0.0      11         1.0                    0   
2845      3.0           0.0       3         1.0                    0   
2846      4.0           0.0       8         1.0                    0   
...       ...           ...     ...         ...                  ...   
3941      2.0           0.0      11         1.0                    0   
3942      6.0           0.0       1         1.0                    0   
3943      3.0           0.0       1         1.0                    0   
3944      3.0           1.0      11         1.0                    0   
3945      3.0           1.0       7         1.0                    1   

      campaign_name_入会費無料  class_name_オールタイム  class_name_デイタイム  gender_F  
2842                    0                  1                

In [20]:
result_test = pd.DataFrame({'y_test':y_test, 'y_pred':y_test_pred})
print(result_test)

      y_test  y_pred
780      1.0     0.0
1808     0.0     1.0
460      1.0     1.0
264      1.0     1.0
161      1.0     1.0
...      ...     ...
590      1.0     1.0
103      1.0     1.0
58       1.0     1.0
1715     0.0     0.0
514      1.0     1.0

[526 rows x 2 columns]


モデルの評価を行う

In [22]:
correct = len(result_test.loc[result_test['y_test'] == result_test['y_pred']])
data_count = len(result_test)
score_test = correct / data_count
print('精度:', score_test)

精度: 0.8916349809885932


In [24]:
print('テスト精度:', model.score(X_test, y_test))
print('学習精度:', model.score(X_train, y_train))

テスト精度: 0.8916349809885932
学習精度: 0.9797211660329531


決定木の深さを変更してモデルの簡易化を行う

In [36]:
X = pd.concat([exit, conti], ignore_index=True) # 学習の説明変数
y = X['is_deleted']  # 学習用の目的変数
del X['is_deleted']  # 学習データには退会結果はいらないので削除
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)  # データを分割

mdoel = DecisionTreeClassifier(random_state=0, max_depth=4)
model.fit(X_train, y_train)
print('テスト精度:', model.score(X_test, y_test))
print('学習精度:', model.score(X_train, y_train))

テスト精度: 0.9068441064638784
学習精度: 0.9803548795944234


モデルに関与している説明変数を確認

In [37]:
importance = pd.DataFrame({'feature_name': X.columns, 'coefficient':model.feature_importances_})
print(importance)

          feature_name  coefficient
0              count_1     0.333898
1         routine_flag     0.111744
2               period     0.454431
3  campaign_name_入会費半額     0.017460
4  campaign_name_入会費無料     0.023853
5    class_name_オールタイム     0.025853
6     class_name_デイタイム     0.011279
7             gender_F     0.021483
